<a href="https://colab.research.google.com/github/mariaher07/FinalProjectDL2020/blob/master/FINAL_PROJECT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **SENTIMENT ANALYSIS: RESTAURANT REVIEWS**
#RNN vs CNN


Authors:

Esther Flores

Candela García

María Hernández

##Preparation of Data

In [27]:


from google.colab import drive
import matplotlib.pyplot as plt
import numpy as np
# Mount Google Drive
drive.mount('/content/drive')
data_path = '/content/drive/My Drive/Deep_Learning2020/FINAL_PROJECT/DATA/'
results_path = '/content/drive/My Drive/Deep_Learning2020/FINAL_PROJECT/RESULTS/'



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Divide dataset

In [28]:
from torchtext import datasets
import pandas

df = pandas.read_csv(data_path+'Restaurant_Reviews.csv',sep='\t')
print(df.head)

from sklearn.model_selection import train_test_split

train, validate, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])



<bound method NDFrame.head of                                                 Review  Liked
0                             Wow... Loved this place.      1
1                                   Crust is not good.      0
2            Not tasty and the texture was just nasty.      0
3    Stopped by during the late May bank holiday of...      1
4    The selection on the menu was great and so wer...      1
..                                                 ...    ...
995  I think food should have flavor and texture an...      0
996                           Appetite instantly gone.      0
997  Overall I was not impressed and would not go b...      0
998  The whole experience was underwhelming, and I ...      0
999  Then, as if I hadn't wasted enough of my life ...      0

[1000 rows x 2 columns]>


In [29]:
print(train)

                                                Review  Liked
38               My first visit to Hiro was a delight!      1
981  We started with the tuna sashimi which was bro...      0
223  Hopefully this bodes for them going out of bus...      0
14   I was disgusted because I was pretty sure that...      0
857  Some may say this buffet is pricey but I think...      1
..                                                 ...    ...
908                             We won't be returning.      0
444  Each day of the week they have a different dea...      1
16                                 Highly recommended.      1
271       You cant go wrong with any of the food here.      1
100  Our server was fantastic and when he found out...      1

[600 rows x 2 columns]


Process sets to create type Dataset

In [30]:
from sklearn.model_selection import train_test_split

from torchtext import data as ttd
from torchtext.data import Example
from torchtext.data import Dataset
import torch
from torchtext import data


In [31]:
def preprocess_data(data):
 
    data.loc[data['Liked'] == 0, ['Liked']] = 'neg'
    data.loc[data['Liked'] == 1, ['Liked']] = 'pos'
    return data

def convert_to_dataset(data, text, label):
    
    list_of_examples = [Example.fromlist(row.tolist(),
                                         fields=[('text', text), ('label', label)]) for _, row in data.iterrows()]


    dataset = Dataset(examples=list_of_examples, fields=[('text', text), ('label', label)])

    return dataset


TEXT = data.Field(tokenize = 'spacy', include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)


train=preprocess_data(train)
validate=preprocess_data(validate)
test=preprocess_data(test)
train_torchtext=convert_to_dataset(train,TEXT,LABEL)
validate_torchtext=convert_to_dataset(validate,TEXT,LABEL)
test_torchtext=convert_to_dataset(test,TEXT,LABEL)



Example of one sequence

In [32]:
print(vars(train_torchtext.examples[0]))

{'text': ['My', 'first', 'visit', 'to', 'Hiro', 'was', 'a', 'delight', '!'], 'label': 'pos'}


In [33]:
MAX_VOCAB_SIZE=25000
TEXT.build_vocab(train_torchtext, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_torchtext)

20 most used words from the vocabulary

In [34]:
print(TEXT.vocab.freqs.most_common(20))

[('.', 489), ('the', 240), (',', 227), ('and', 215), ('I', 209), ('was', 186), ('!', 154), ('a', 146), ('to', 127), ('is', 107), ('The', 102), ('it', 89), ('of', 85), ('place', 73), ('for', 72), ('not', 70), ('this', 69), ('in', 66), ("n't", 65), ('food', 65)]


In [35]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_torchtext, validate_torchtext, test_torchtext), 
    batch_size = 64,
    device = device,
    sort_key= lambda x: len(x.text),
    sort_within_batch=False)

##RNN

###Create the model

In [36]:
import torch.nn as nn
class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text, text_lengths):
        
        embedded = self.dropout(self.embedding(text))
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths,enforce_sorted=False)
        
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        

        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
    
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
                
            
        return self.fc(hidden)

In [37]:

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 32
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)


In [38]:
#embeddings setting
pretrained_embeddings = TEXT.vocab.vectors

print(pretrained_embeddings.shape)

model.embedding.weight.data.copy_(pretrained_embeddings)
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)

torch.Size([1753, 100])
tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [-1.4103, -0.2549, -0.1962,  ...,  0.2221,  0.8597,  0.3654],
        [ 0.2911,  0.1218, -0.1307,  ...,  0.2353,  0.3025, -0.0789],
        [ 0.1095,  0.1958,  0.5703,  ..., -0.0667, -0.3019,  0.4945]])


###Train the model

In [39]:
# Train function
import torch.optim as optim

# Device configuration (choose GPU if it is available )

optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)


In [40]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc


In [41]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [42]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
        print(iterator)
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [43]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [44]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), results_path + '/'+'RNN-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.690 | Train Acc: 53.75%
	 Val. Loss: 0.684 |  Val. Acc: 55.86%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.685 | Train Acc: 53.70%
	 Val. Loss: 0.680 |  Val. Acc: 55.47%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.686 | Train Acc: 54.90%
	 Val. Loss: 0.677 |  Val. Acc: 55.86%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.679 | Train Acc: 55.16%
	 Val. Loss: 0.673 |  Val. Acc: 58.20%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.672 | Train Acc: 60.00%
	 Val. Loss: 0.665 |  Val. Acc: 61.72%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.650 | Train Acc: 66.67%
	 Val. Loss: 0.655 |  Val. Acc: 64.06%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.629 | Train Acc: 69.01%
	 Val. Loss: 0.650 |  Val. Acc: 65.23%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.603 | Train Acc: 70.52%
	 Val. Loss: 0.623 |  Val. Acc: 66.02%
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.562 | Train Acc: 73.28%
	 Val. Loss: 0.615 |  Val. Acc: 71.88%
Epoch: 10 | Epoch Time: 0m 0

In [45]:
model.load_state_dict(torch.load(results_path + '/'+'RNN-model.pt'))

test_loss, test_acc = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

Test Loss: 0.620 | Test Acc: 70.31%


Final result under 0.5 ==> negative review

Final result above 0.5 ==> positive review

In [47]:
import spacy
nlp = spacy.load('en')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    if list(LABEL.vocab.stoi.keys())[0] == "neg":
      my_pred = prediction.item()
    else:
      my_pred = 1-prediction.item()
    if my_pred> 0.5:
        print("Positive review")
    else:
        print("Negative review")
    return my_pred

    

We try 4 different sentences, which sentiment is predicted correctly


In [48]:
predict_sentiment(model, "I probably won't be coming back here.")

Negative review


0.11246383190155029

In [49]:
predict_sentiment(model, "The chips and sals a here is amazing!!!!!!!!!!!!!!!!!!!")

Positive review


0.8823733925819397

In [50]:
predict_sentiment(model, "I hated it")

Negative review


0.48276200890541077

In [51]:
predict_sentiment(model, "It sucks")

Negative review


0.45071691274642944

Number of parameters around 230k

In [53]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 234,757 trainable parameters


##CNN

###Preprocessing the data

In [54]:
TEXT_CNN = data.Field(tokenize = 'spacy', batch_first = True)
LABEL_CNN = data.LabelField(dtype = torch.float)

In [55]:
from torchtext import datasets
import pandas

df_cnn = pandas.read_csv(data_path+'Restaurant_Reviews.csv',sep='\t')

train_cnn, validate_cnn, test_cnn = np.split(df_cnn.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
train_cnn=preprocess_data(train_cnn)
validate_cnn=preprocess_data(validate_cnn)
test_cnn=preprocess_data(test_cnn)
train_torchtext_cnn=convert_to_dataset(train_cnn,TEXT_CNN,LABEL_CNN)
validate_torchtext_cnn=convert_to_dataset(validate_cnn,TEXT_CNN,LABEL_CNN)
test_torchtext_cnn=convert_to_dataset(test_cnn,TEXT_CNN,LABEL_CNN)

In [56]:
MAX_VOCAB_SIZE = 25_000

TEXT_CNN.build_vocab(train_torchtext_cnn, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL_CNN.build_vocab(train_torchtext_cnn)

In [57]:
BATCH_SIZE = 64

device_cnn = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator_cnn, valid_iterator_cnn, test_iterator_cnn = data.BucketIterator.splits(
    (train_torchtext_cnn, validate_torchtext_cnn, test_torchtext_cnn), 
    batch_size = BATCH_SIZE, 
    device = device_cnn,
    sort_key= lambda x: len(x.text),
    sort_within_batch=False)

###Create the model

In [60]:
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
                
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        embedded = self.embedding(text)
        
        embedded = embedded.unsqueeze(1)
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
                
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        cat = self.dropout(torch.cat(pooled, dim = 1))

        return self.fc(cat)

In [61]:
INPUT_DIM_cnn = len(TEXT_CNN.vocab)
EMBEDDING_DIM_cnn = 100
N_FILTERS_cnn = 100
FILTER_SIZES_cnn = [3,4,5]
OUTPUT_DIM_cnn = 1
DROPOUT_cnn = 0.5
PAD_IDX_cnn = TEXT_CNN.vocab.stoi[TEXT_CNN.pad_token]

model_cnn = CNN(INPUT_DIM_cnn, EMBEDDING_DIM_cnn, N_FILTERS_cnn, FILTER_SIZES_cnn, OUTPUT_DIM_cnn, DROPOUT_cnn, PAD_IDX_cnn)

In [62]:
#embeddings setting
pretrained_embeddings_cnn = TEXT_CNN.vocab.vectors

model_cnn.embedding.weight.data.copy_(pretrained_embeddings_cnn)

tensor([[-0.8034, -1.3328,  1.9760,  ...,  0.9751,  1.8569,  0.1438],
        [-0.2599, -0.9822, -0.6308,  ...,  0.1010, -0.3009, -0.3985],
        [-0.3398,  0.2094,  0.4635,  ..., -0.2339,  0.4730, -0.0288],
        ...,
        [ 2.1339,  0.3628,  0.1081,  ..., -1.7391,  0.8406,  0.7634],
        [-0.5936, -0.0749,  0.5304,  ...,  0.2310,  0.5684,  0.4886],
        [-0.4735, -0.4899,  1.3609,  ...,  0.6387,  0.2690, -0.0217]])

In [63]:
UNK_IDX_cnn = TEXT_CNN.vocab.stoi[TEXT_CNN.unk_token]

model_cnn.embedding.weight.data[UNK_IDX_cnn] = torch.zeros(EMBEDDING_DIM_cnn)
model_cnn.embedding.weight.data[PAD_IDX_cnn] = torch.zeros(EMBEDDING_DIM_cnn)

###Train the model

In [64]:
import torch.optim as optim

optimizer_cnn = optim.Adam(model_cnn.parameters())

criterion_cnn = nn.BCEWithLogitsLoss()

model_cnn = model_cnn.to(device_cnn)
criterion_cnn = criterion_cnn.to(device_cnn)

In [65]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.text).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [66]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.text).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [67]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model_cnn, train_iterator_cnn, optimizer_cnn, criterion_cnn)
    valid_loss, valid_acc = evaluate(model_cnn, valid_iterator_cnn, criterion_cnn)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model_cnn.state_dict(), results_path + '/'+'cnn-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 0m 0s
	Train Loss: 0.667 | Train Acc: 59.17%
	 Val. Loss: 0.658 |  Val. Acc: 64.84%
Epoch: 02 | Epoch Time: 0m 0s
	Train Loss: 0.578 | Train Acc: 73.59%
	 Val. Loss: 0.641 |  Val. Acc: 64.06%
Epoch: 03 | Epoch Time: 0m 0s
	Train Loss: 0.518 | Train Acc: 79.27%
	 Val. Loss: 0.624 |  Val. Acc: 66.41%
Epoch: 04 | Epoch Time: 0m 0s
	Train Loss: 0.447 | Train Acc: 85.36%
	 Val. Loss: 0.612 |  Val. Acc: 70.31%
Epoch: 05 | Epoch Time: 0m 0s
	Train Loss: 0.391 | Train Acc: 87.76%
	 Val. Loss: 0.579 |  Val. Acc: 71.09%
Epoch: 06 | Epoch Time: 0m 0s
	Train Loss: 0.320 | Train Acc: 91.98%
	 Val. Loss: 0.571 |  Val. Acc: 71.48%
Epoch: 07 | Epoch Time: 0m 0s
	Train Loss: 0.282 | Train Acc: 93.54%
	 Val. Loss: 0.555 |  Val. Acc: 73.05%
Epoch: 08 | Epoch Time: 0m 0s
	Train Loss: 0.228 | Train Acc: 94.69%
	 Val. Loss: 0.535 |  Val. Acc: 73.83%
Epoch: 09 | Epoch Time: 0m 0s
	Train Loss: 0.178 | Train Acc: 96.46%
	 Val. Loss: 0.521 |  Val. Acc: 74.61%
Epoch: 10 | Epoch Time: 0m 0

In [68]:
model_cnn.load_state_dict(torch.load(results_path + '/'+'cnn-model.pt'))

test_loss_cnn, test_acc_cnn = evaluate(model_cnn, test_iterator_cnn, criterion_cnn)

print(f'Test Loss: {test_loss_cnn:.3f} | Test Acc: {test_acc_cnn*100:.2f}%')

Test Loss: 0.545 | Test Acc: 75.00%


In [69]:
nlp = spacy.load('en')

def predict_sentiment(model, sentence, min_len = 5):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT_CNN.vocab.stoi[t] for t in tokenized]
    tensor = torch.LongTensor(indexed).to(device_cnn)
    tensor = tensor.unsqueeze(0)
    prediction = torch.sigmoid(model(tensor))
    if list(LABEL_CNN.vocab.stoi.keys())[0] == "neg":
      my_pred = prediction.item()
    else:
      my_pred = 1-prediction.item()
    if my_pred> 0.5:
        print("Positive review")
    else:
        print("Negative review")
    return my_pred

We try the 4 same sentences as for RNN

In [71]:
predict_sentiment(model_cnn, "I probably won't be coming back here.")

Negative review


0.08322006464004517

In [72]:
predict_sentiment(model_cnn, "The chips and sals a here is amazing!!!!!!!!!!!!!!!!!!!")

Positive review


0.9719551559537649

In [73]:
predict_sentiment(model_cnn, "I hated it")

Negative review


0.32960784435272217

In [74]:
predict_sentiment(model_cnn, "It sucks")

Negative review


0.17209982872009277

Parameters of the model around 280k

In [75]:
print(f'The model has {count_parameters(model_cnn):,} trainable parameters')

The model has 288,001 trainable parameters
